<a href="https://colab.research.google.com/github/isabellacadisco/textsent-unimi-2023/blob/master/gibbs_sampling_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial on collapsed Gibbs sampling

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

docs = [
    ['apple', 'ios', 'mac', 'book'],
    ['apple', 'mac', 'book', 'apple', 'store'],
    ['mac', 'book', 'ios', 'store'],
    ['banana', 'mango', 'fruit'],
    ['apple', 'fruit'],
    ['orange', 'strawberry', 'banana'],
    ['orange', 'mango', 'banana'],
    ['fruit', 'apple', 'mac', 'ios']
]
K = 2
words = list(set([x for y in docs for x in y]))
alpha = np.array([1]*K)
beta = np.array([1]*len(words))

toy example
problem w/ this approach: set up number of topics, parameter k
words=vocabolary set of all the words list comprehension
parameters alpha beta

## Init counters of assignments
- $n_{d,k}$ number of words of the document $d$ assigned to $k$
- $n_{k,w}$ number of times (instances of) word $w$ is assigned to $k$
- $n_k$ number of word instances assignements to $k$
- $z$ array of assignments to topics for each of the words instances

some quantities we are looking for, global n = total number of words, z array over the topics for all occurrencies of words position in this array that is 0, n_{dk} rows=number of topics columns=number of docs (forse è il contrario)
n_{kw} rows=topics columns=words, occurencies of
n_{k} vector same size fo topics

In [2]:
N = sum([len(x) for x in docs])
z = np.zeros(N)
ndk = np.zeros((len(docs), K))
nkw = np.zeros((K, len(words)))
nk = np.zeros(K)

### Randomly initialize

random allocation
over all docs, all the words in doc, append the occcurencies that we observe, define random choice in range 0 k (pick up random topic), allocate in z matrix (vector same size of the words), basically we are teh same as before with the table but over words, 

count number of times im oberserving that topic, n_{dk} of i (columns of the topic) put the number of time i observe that topic, n{kw} index in the voc of that word (column index), row of topic adn put +1

just counting

In [3]:
instance = 0
occurrences = []
for i, doc in enumerate(docs):
    for w in doc:
        occurrences.append(w)
        t = np.random.choice(range(0,K))
        z[instance] = t
        nk[t] += 1
        ndk[i,t] += 1
        nkw[t, words.index(w)] += 1
        instance += 1

In [4]:
display(pd.DataFrame(z, index=occurrences).T)
display(pd.DataFrame(ndk, index=range(len(docs)), columns=range(K)))
display(pd.DataFrame(nkw, index=range(K), columns=words))

,apple,ios,mac,book,apple,mac,book,apple,store,mac,...,orange,strawberry,banana,orange,mango,banana,fruit,apple,mac,ios
0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0


,0,1
0,0.0,4.0
1,3.0,2.0
2,2.0,2.0
3,1.0,2.0
4,2.0,0.0
5,2.0,1.0
6,2.0,1.0
7,1.0,3.0


,ios,store,apple,book,orange,mac,mango,banana,fruit,strawberry
0,2.0,1.0,2.0,0.0,1.0,2.0,1.0,2.0,1.0,1.0
1,1.0,1.0,3.0,3.0,1.0,2.0,1.0,1.0,2.0,0.0


z = vector over words with topic, topic is 0 or 1

n_{dk} = prima linea, 0 in topic 0, 4 in topic 1, p(docs over tooics)

n_{kw} = ios put 2 times n topic 0 and 1 in topic 1, probability (if normalize) of the words over the topics

what is the most important word in topic zero? 

## Likelihood

transform these numbers in probabilities, theta and phi conditioned by alpha

In [5]:
def theta(ndk, alpha, d_i, k_i):
    return (ndk[d_i, k_i] + alpha[k_i]) / (np.sum(ndk[d_i,:]) + alpha[k_i])

def phi(nkw, beta, w_i, k_i):
    return (nkw[k_i, w_i] + beta[w_i]) / (np.sum(nkw[k_i,:] + beta))

example, take theta of something

In [6]:
print(theta(ndk, alpha, 1, 0))

0.6666666666666666


# Gibbs Sampling

for each docs
versione nuova con commenti su github

In [16]:
def gibbs(docs, words, topics, ndk, nkw, nk, alpha, beta, iterations=10):
    for it in range(0, iterations):
        w_i = 0 #initially set to zero
        for doc_i, doc in enumerate(docs): #go over docs
            for w in doc: #for each word in doc
                word = [x for y in docs for x in y][w_i] #take word out of our doc, si può fare fuori dal ciclo in realtà volendo
                topic = int(topics[w_i]) #get the topic, transform into integer cause just 2
                p_z = np.zeros(len(alpha)) #create a list of probabilities
                for k_i in range(0, len(alpha)):
                    p_z[k_i] = theta(ndk, alpha, doc_i, k_i) * phi(nkw, beta, words.index(word), k_i)
                p_z = p_z / np.sum(p_z)
                # Sample from p_z
                new_topic = np.random.choice(len(p_z), 1, p=p_z)[0] # i want to use these probablities to sample a new topic, extractin new topic for word a with those prob
                # Update
                # Remove current assignment
                ndk[doc_i, topic] -= 1
                nkw[topic, words.index(word)] -= 1
                nk[topic] -= 1
                topics[w_i] = new_topic
                ndk[doc_i, new_topic] += 1
                nkw[new_topic, words.index(word)] += 1
                nk[new_topic] += 1
                w_i += 1
#store matrices every time, history of updating

In [17]:
gibbs(docs, words, z, ndk, nkw, nk, alpha, beta, iterations=1000)

the following is the final output, 
first vector=topics to word
third = words to topic

In [18]:
display(pd.DataFrame(z, index=occurrences).T)
display(pd.DataFrame(ndk, index=range(len(docs)), columns=range(K)))
display(pd.DataFrame(nkw, index=range(K), columns=words))

,apple,ios,mac,book,apple,mac,book,apple,store,mac,...,orange,strawberry,banana,orange,mango,banana,fruit,apple,mac,ios
0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0


,0,1
0,2.0,2.0
1,3.0,2.0
2,3.0,1.0
3,2.0,1.0
4,1.0,1.0
5,2.0,1.0
6,0.0,3.0
7,3.0,1.0


,ios,store,apple,book,orange,mac,mango,banana,fruit,strawberry
0,1.0,2.0,3.0,0.0,1.0,4.0,0.0,1.0,3.0,1.0
1,2.0,0.0,2.0,3.0,1.0,0.0,2.0,2.0,0.0,0.0


# Sklearn implementation

In [19]:
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter


M = np.zeros((len(words), len(docs)))
for i, doc in enumerate(docs):
    for k, w in Counter(doc).most_common():
        M[words.index(k), i] = w
LDA = LatentDirichletAllocation(n_components=2, learning_method='batch', max_iter=100).fit(M.T)

In [20]:
A = LDA.components_

In [21]:
A.T

array([[3.48999939, 0.51000061],
       [2.49166986, 0.50833014],
       [5.48683864, 0.51316136],
       [3.4913286 , 0.5086714 ],
       [0.5039721 , 2.4960279 ],
       [4.49064471, 0.50935529],
       [0.50460803, 2.49539197],
       [0.50442721, 3.49557279],
       [2.41387108, 1.58612892],
       [0.50388518, 1.49611482]])

Topic word distribution. components_[i, j] represents word j in topic i.

In [22]:
display(pd.DataFrame(A, index=range(K), columns=words))

,ios,store,apple,book,orange,mac,mango,banana,fruit,strawberry
0,3.489999,2.49167,5.486839,3.491329,0.503972,4.490645,0.504608,0.504427,2.413871,0.503885
1,0.510001,0.50833,0.513161,0.508671,2.496028,0.509355,2.495392,3.495573,1.586129,1.496115


In [23]:
Terms = np.zeros((len(words), K))
Docs = np.zeros((len(docs), K))

for w_i, w in enumerate(words):
    Terms[w_i] = A.T[w_i] / np.sum(A.T[w_i])

for d_i, d in enumerate(docs):
    dM = np.zeros((len(d), K))
    for j, w in enumerate(d):
        dM[j] = Terms[words.index(w)]
    Docs[d_i] = np.sum(dM, axis=0) / np.array([len(d)]*K)

allocation of words over the topics
of topics over docs

In [24]:
display(pd.DataFrame(Terms, index=words, columns=range(0,K)))
display(pd.DataFrame(Docs, index=range(0, len(docs)), columns=range(0,K)))

,0,1
ios,0.872500,0.127500
store,0.830557,0.169443
apple,0.914473,0.085527
book,0.872832,0.127168
orange,0.167991,0.832009
mac,0.898129,0.101871
mango,0.168203,0.831797
banana,0.126107,0.873893
fruit,0.603468,0.396532
strawberry,0.251943,0.748057


,0,1
0,0.889484,0.110516
1,0.886093,0.113907
2,0.868504,0.131496
3,0.299259,0.700741
4,0.758970,0.241030
5,0.182013,0.817987
6,0.154100,0.845900
7,0.822142,0.177858


In [25]:
sigma_w, sigma_d = 4, 3
for t_k in range(0, K):
    print('topic', t_k)
    top_w = sorted([(i, wk) for i, wk in enumerate(Terms.T[t_k])], key=lambda y: -y[1])[:sigma_w]
    top_d = sorted([(i, dk) for i, dk in enumerate(Docs.T[t_k])], key=lambda y: -y[1])[:sigma_d]
    print([words[x[0]] for x in top_w])
    print([docs[x[0]] for x in top_d])
    print("")

topic 0
['apple', 'mac', 'book', 'ios']
[['apple', 'ios', 'mac', 'book'], ['apple', 'mac', 'book', 'apple', 'store'], ['mac', 'book', 'ios', 'store']]

topic 1
['banana', 'orange', 'mango', 'strawberry']
[['orange', 'mango', 'banana'], ['orange', 'strawberry', 'banana'], ['banana', 'mango', 'fruit']]



why is working?
cause im randomly updating matrices, everythung is depending on counting of words in docs, when i transform these counts into prob just normalizing, if words apprearing lot in doc push doc over topic, i have estimation of topics, use this estimation to extract new topic

document is the glue between these two prob 